# test

## include scripts

### include notebook core

In [ ]:
. ../../../../scripts/nbs_header.ps1

### Include core functions script

In [ ]:
. ../../../../scripts/core.ps1

### Include spiral library

In [ ]:
. ../../../../lib/spiral/lib.ps1

## execute project commands

### run notebook with retries using spiral supervisor

In [ ]:
{ . ../../../../apps/spiral/dist/Supervisor$(_exe) --execute-command "../../../../target/release/spiral_builder$(_exe) dib --path test.dib --retries 3" } | Invoke-Block

00:00:00   debug #1 run_with_timeout_async / timeout: 500
00:00:01   debug #1 execute_with_options_async / options: struct (Some System.Threading.CancellationToken,
        "../../../../target/release/spiral_builder.exe dib --path test.dib --retries 3",
        [||], None, None, None)
00:00:01 verbose #2 > 00:00:00   debug #1 spiral_builder.main / args: MutCell(["dib", "--path", "test.dib", "--retries", "3"])
00:00:01 verbose #3 > 00:00:00   debug #2 runtime.execute_with_options / file_name: dotnet / arguments: ["repl", "--exit-after-run", "--run", "c:/home/git/polyglot/apps/spiral/temp/test/test.dib", "--output-path", "c:/home/git/polyglot/apps/spiral/temp/test/test.dib.ipynb"] / options: (None, "dotnet repl --exit-after-run --run "c:/home/git/polyglot/apps/spiral/temp/test/test.dib" --output-path "c:/home/git/polyglot/apps/spiral/temp/test/test.dib.ipynb"", Array(MutCell([("AUTOMATION", "True")])), None, None, None)
00:00:11 verbose #4 > 00:00:09 verbose #3 >
00:00:11 verbose #5 > 00

### parse the .dib file into .spi format with dibparser

In [ ]:
{ . ../../../../apps/parser/dist/DibParser$(_exe) test.dib spi } | Invoke-Block

00:00:00   debug #1 writeDibCode / output: Spi / path: test.dib
00:00:00   debug #2 parseDibCode / output: Spi / file: test.dib


### build .fsx file from .spi using supervisor

In [ ]:
{ . ../../../../apps/spiral/dist/Supervisor$(_exe) --build-file test.spi test.fsx } | Invoke-Block

00:00:00   debug #1 run_with_timeout_async / timeout: 500
00:00:01   debug #2 run_with_timeout_async / timeout: 500
00:00:01   debug #1 buildFile / takeWhileInclusive / path: test.spi / fsxContent:  / errors: [] / typeErrorCount: 0
00:00:01   debug #2 buildFile / takeWhileInclusive / path: test.spi / fsxContent:  / errors: [] / typeErrorCount: 0
00:00:01 verbose #3 sendJson / port: 13805 / json: {"FileOpen":{"spiText":"/// # test (Polyglot)\nnominal i = ()\nnominal e = ()\nnominal s = ()\nnomin...0022\u003C/test\u003E\u0022\n","uri":"file:///c:/home/git/polyglot/apps/spiral/temp/test/test.spi"}} / result:
00:00:01 verbose #4 sendJson / port: 13805 / json: {"BuildFile":{"backend":"Fsharp","uri":"file:///c:/home/git/polyglot/apps/spiral/temp/test/test.spi"}} / result:
00:00:02   debug #5 buildFile / takeWhileInclusive / path: test.spi / fsxContent:  / errors: [] / typeErrorCount: 0
00:00:03   debug #6 buildFile / takeWhileInclusive / path: test.spi / fsxContent:  / errors: [] / typeError

## compile and format the project

### compile project with fable targeting optimized rust

In [ ]:
dotnet fable --optimize --lang rs --extension .rs

Fable 4.17.0: F# to Rust compiler (status: alpha)

Thanks to the contributor! @alfonsogarciacaro
Stand with Ukraine! https://standwithukraine.com.ua/

Parsing test.fsproj...
.> cmd /C dotnet restore test.fable-temp.csproj -p:FABLE_COMPILER=true -p:FABLE_COMPILER_4=true -p:FABLE_COMPILER_RUST=true
  Determining projects to restore...
  Restored C:\home\git\polyglot\apps\spiral\temp\test\test.fable-temp.csproj (in 1.28 sec).
.> cmd /C dotnet restore C:/home/git/polyglot/apps/spiral/temp/test/test.fsproj
  Determining projects to restore...
  Restored C:\home\git\polyglot\apps\spiral\temp\test\test.fsproj (in 324 ms).
Project and references (1 source files) parsed in 8216ms

Started Fable compilation...

Fable compilation finished in 1584ms

.\test.fsx(6,0): (6,2) warning FABLE: For Rust, support for F# static and module do bindings is disabled by default. It can be enabled with the 'static_do_bindings' feature. Use at your own risk!


### fix formatting issues in the .rs file using regex and set-content

In [ ]:
(Get-Content test.rs) `
    -replace [regex]::Escape("),);"), "));" `
    | FixRust `
| Set-Content test.rs

### format the rust code using cargo fmt

In [ ]:
cargo fmt --

## build and test the project

### build the project in release mode using nightly rust compiler

In [ ]:
cargo +nightly build --release

   Compiling num-traits v0.2.19
   Compiling getrandom v0.2.15
   Compiling libm v0.2.8
   Compiling windows-sys v0.52.0
   Compiling fastrand v2.1.0
   Compiling ppv-lite86 v0.2.17
   Compiling bit-vec v0.6.3
   Compiling quick-error v1.2.3
   Compiling rand_core v0.6.4
   Compiling wait-timeout v0.2.0
   Compiling thiserror v1.0.60
   Compiling fnv v1.0.7
   Compiling rand_xorshift v0.3.0
   Compiling rand_chacha v0.3.1
   Compiling bit-set v0.5.3
   Compiling uuid v1.8.0
   Compiling thiserror-impl v1.0.60
   Compiling rand v0.8.5
   Compiling lazy_static v1.4.0
   Compiling minimal-lexical v0.2.1
   Compiling tempfile v3.10.1
   Compiling unarray v0.1.4
   Compiling bitflags v2.5.0
   Compiling nom v7.1.3
   Compiling rusty-fork v0.3.0
   Compiling chrono v0.4.38
   Compiling proptest v1.4.0
   Compiling fable_library_rust v0.1.0 (C:\home\git\polyglot\lib\rust\fable\fable_modules\fable-library-rust)
   Compiling spiral_temp_test v0.0.1 (C:\home\git\polyglot\apps\spiral\temp\test)
 

### run release tests with output enabled

In [ ]:
{ cargo +nightly test --release -- --show-output } | Invoke-Block

   Compiling num-traits v0.2.19
   Compiling fastrand v2.1.0
   Compiling windows-sys v0.52.0
   Compiling fnv v1.0.7
   Compiling bit-vec v0.6.3
   Compiling wait-timeout v0.2.0
   Compiling quick-error v1.2.3
   Compiling futures-util v0.3.30
   Compiling rand v0.8.5
   Compiling rand_xorshift v0.3.0
   Compiling minimal-lexical v0.2.1
   Compiling bit-set v0.5.3
   Compiling unarray v0.1.4
   Compiling bitflags v2.5.0
   Compiling nom v7.1.3
   Compiling thiserror v1.0.60
   Compiling chrono v0.4.38
   Compiling tempfile v3.10.1
   Compiling rusty-fork v0.3.0
   Compiling proptest v1.4.0
   Compiling futures-executor v0.3.30
   Compiling futures v0.3.30
   Compiling fable_library_rust v0.1.0 (C:\home\git\polyglot\lib\rust\fable\fable_modules\fable-library-rust)
   Compiling spiral_temp_test v0.0.1 (C:\home\git\polyglot\apps\spiral\temp\test)
    Finished `release` profile [optimized] target(s) in 34.19s
     Running unittests main.rs (C:\home\git\polyglot\target\release\deps\spiral_

### execute the binary in release mode

In [ ]:
{ . $ScriptDir/../../../../target/release/spiral_temp_test$(_exe) } | Invoke-Block

app=test
